In [ ]:
using Flux, Statistics, Plots, StatsBase, MAT, LinearAlgebra, CUDA, ProgressMeter, Random, MultivariateStats, MLUtils

In [ ]:
CUDA.device!(1)

In [ ]:
gt = matread("HSI/KSC/KSC_gt.mat")["KSC_gt"];           #loading ground truth coordinates, enter location according to your computer

In [ ]:
indexData = Dict();

In [ ]:
for i in 1:13
    indexData[i] = findall(x->x==i,gt)
end

In [ ]:
for i in 1:13
    indexData[i] = indexData[i][shuffle(1:length(indexData[i]))]
end

In [ ]:
alldata = matread("HSI/KSC/KSC.mat")["KSC"]./500; #loading and normalizing the hyperspectral image, 500 was randomly chosen as threshold to discard pixels with unnaturally high values/data artifects, see later

In [ ]:
bpx = Dict() #for storing coordinates of bad pixels we discard before training
gpx = Dict() #for storing coordinates of good pixels we use for training

In [ ]:
for i in 1:13
    bpx[i]=[]
    gpx[i]=[]
end

In [ ]:
for i in 1:13
    for j in 1:length(indexData[i])
        if (maximum(alldata[indexData[i][j][1],indexData[i][j][2],:])>=1)   #using threshold to separate bad pixels
            push!(bpx[i],(indexData[i][j][1],indexData[i][j][2]))
        else
            push!(gpx[i],(indexData[i][j][1],indexData[i][j][2]))
        end
    end
end

In [ ]:
trainindex = [gpx[i] for i in 1:13]

In [ ]:
testindex = []
for i in 1:13
    m = floor(Int,size(trainindex[i])[1]*0.1)               #separating data into 1:9 train-test split
    push!(testindex,trainindex[i][1:end-m])
    trainindex[i]= trainindex[i][end-m+1:end]
end

In [ ]:
trainvector=[Array{Float64}(undef,176,length(trainindex[i])) for i in 1:13]

In [ ]:
for i in 1:13
    for j in 1:length(trainindex[i])
        trainvector[i][:,j]=alldata[trainindex[i][j][1],trainindex[i][j][2],:]
    end
end

In [ ]:
testvector=[Array{Float64}(undef,176,length(testindex[i])) for i in 1:13]

In [ ]:
for i in 1:13
    for j in 1:length(testindex[i])
        testvector[i][:,j]=alldata[testindex[i][j][1],testindex[i][j][2],:]
    end
end

# SymAE's code:

In [ ]:
function Split(x::AbstractArray)                            #We'll use these functions in the architecture
    reshape(x, (div(size(x)[1], nτ),nτ,size(x)[2]))
end

function UnSplit(x::AbstractArray)
    reshape(x,(176*nτ,:))                       #176 is number of spectral bands in KSC scene
end

In [ ]:
nc = 64             # reflectance code dimension
nd = 64             # nuisance code dimension
nτ = 8              # number of pixels in a datapoint

In [ ]:
Dr1 = Dropout(0.25)
Dr2 = Dropout(0.5)
renc = Chain(Dense(176,300,x->leakyrelu(x,0.5)),Dense(300,300,x->leakyrelu(x,0.5)),Dense(300,300,x->leakyrelu(x,0.5)),Dense(300,150,x->leakyrelu(x,0.5)),Dense(150,nc,x->leakyrelu(x,0.5))) |> gpu
nenc = Chain(Dense(176,300,x->leakyrelu(x,0.5)),Dr1,Dense(300,300,x->leakyrelu(x,0.5)),Dense(300,300,x->leakyrelu(x,0.5)),Dr1,Dense(300,150,x->leakyrelu(x,0.5)),Dense(150,nd,x->leakyrelu(x,0.5))) |>gpu
dec = Chain(Dense(nc+nd,150,x->leakyrelu(x,0.5)),Dense(150,300,x->leakyrelu(x,0.5)),Dense(300,300,x->leakyrelu(x,0.5)),Dense(300,300,x->leakyrelu(x,0.5)),Dense(300,300,x->leakyrelu(x,0.5)),Dense(300,300,x->leakyrelu(x,0.5)),Dense(300,176)) |>gpu
SYMAE = Chain(Parallel((x,y)->cat(x,y,dims=1),Chain(x->Split(x),renc,x->mean(x,dims=2),x->fill(x,nτ),x->cat(x...,dims=2)),Chain(x->Split(x),nenc,Dr2)),dec,x->UnSplit(x)) |> gpu


In [ ]:
L(x) = Flux.Losses.mse(SYMAE(x),x) |>gpu
ps = Flux.params(SYMAE) |>gpu
opt = ADAM(0.0001) |>gpu

In [ ]:
###############################################################

function generate_samples(dvec, n=1000, ntau=nτ)                          #function to generate random datapoints and batch them for training
    data=[]
    for i in 1:n
        push!(data,Flux.batch([vec(randobs(randobs(dvec), ntau)) for i = 1:16*2*8]))
    end
    return gpu(data)
end

################################################################

In [ ]:
trainloss_store=[]

In [ ]:
dtrain = generate_samples(trainvector,2*1024);  #storing data for training

In [ ]:
@time tr0 = mean(Flux.Losses.mse.(SYMAE.(dtrain),dtrain)) #initial train loss

In [ ]:
nepoch=1000
ProgressMeter.ijulia_behavior(:clear)
p=Progress(nepoch)
for epoch in 1:nepoch
    for x in dtrain
        # x, y = gpu(x), gpu(y) # transfer data to device
        gs = gradient(() -> Flux.mse(SYMAE(x), x), ps) # compute gradient
        Flux.Optimise.update!(opt, ps, gs) # update parameters
    end
    train_loss=mean(Flux.Losses.mse.(SYMAE.(dtrain),dtrain))
    push!(trainloss_store, train_loss)
    ProgressMeter.next!(p; showvalues = [(:epoch,epoch),(:train_loss,train_loss)])
end

In [ ]:
plot(1:length(trainloss_store),trainloss_store,label = false)

In [ ]:
renc = SYMAE[1][1][2];

In [ ]:
nenc = SYMAE[1][2][2];

In [ ]:
dec = SYMAE[2];

In [ ]:
j = rand(collect(1:length(trainvector)))
ab = randobs(trainvector1[j],10);
nuis = nenc(gpu(ab[:,1]));
coh = renc(gpu(ab));
mmm=Flux.stack([[1.0] for i in 1:10],dims = 2);
nnn=nuis*gpu(mmm);
kkk =vcat(coh,nnn);
lll = dec(kkk);
p1 = plot(1:176,ab,title= "Not-Redatumed",label=false)
p2 = plot(1:176,cpu(lll),title="Redatumed",label=false)
plot!(p1,p2,layout=(2,1))

In [ ]:
j = rand(collect(1:length(testvector)))
ab = randobs(testvector1[j],10);
nuis = nenc(gpu(ab[:,1]));
coh = renc(gpu(ab));
mmm=Flux.stack([[1.0] for i in 1:10],dims = 2);
nnn=nuis*gpu(mmm);
kkk =vcat(coh,nnn);
lll = dec(kkk);
p1 = plot(1:176,ab,title= "Not-Redatumed",label=false)
p2 = plot(1:176,cpu(lll),title="Redatumed",label=false)
plot!(p1,p2,layout=(2,1))

# The configuration of Feed Forward Network used for HSI classification:

In [ ]:
model = Chain(Dense(nc,8*128,leakyrelu),Dense(8*128,4*128,leakyrelu),Dropout(0.5),Dense(4*128,64,leakyrelu),Dense(64,13,identity),softmax) |> gpu
L(x,y) = Flux.Losses.crossentropy(model(x),y)
ps = Flux.params(model) |> gpu;
opt = ADAM(0.0001)